# import packages

In [1]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
#import nltk
import string
from gensim.corpora import Dictionary
# preprocessing imports
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
# model imports
from gensim.utils import simple_preprocess

from gensim.models import CoherenceModel, Phrases, phrases
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from gensim.test.utils import get_tmpfile
from sklearn.preprocessing import normalize

#import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

# read tables from mysql

In [ ]:
printable = set(string.printable)
s = 'Ti\u00ebsto'
list(filter(lambda x: x in printable, s))

In [ ]:
# read lyrics.csv
data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])

In [ ]:
def preprocess(text):
    try:
        return simple_preprocess(text)
    except Exception as e:
        print(str(e))
        return np.nan

# searching using d2v

In [ ]:
#lyrics['singer'] = list(map(simple_preprocess, lyrics.singer))
#lyrics['song'] = list(map(simple_preprocess, lyrics.singer))
#data['lyrics'] = list(map(preprocess, data.lyrics.values))
data = pd.read_csv('lyrics.csv', names = ['singer', 'song', 'lyrics'])
data = data[~data['lyrics'].isna()]
data = data[~data['song'].isna()]

In [ ]:
def infer_w2v_features(d2v_model, sentence):
    #vector = d2v_model.docvecs[i]
        
    inferred_vector = d2v_model.infer_vector(sentence)
    return inferred_vector

In [ ]:
fname = get_tmpfile("my_doc2vec_model")
model = Doc2Vec.load(fname)

In [ ]:
lyric = 'Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our ton the best Pride and joy and dirty dreams'
inferred_vector = model.infer_vector(lyric.split(' '))
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
rank = [[data.index.get_loc(int(docid.strip('doc'))), sim] for docid, sim in sims]
#rank = [((int(docid.strip('doc'))), sim) for docid, sim in sims]


In [ ]:
rank_df = pd.DataFrame(rank, columns=['docid', 'sim'])
rank_df = rank_df.sort_values(by=['docid'])
rank_df.index = rank_df.docid
rank_df = rank_df.drop(columns=['docid'])

In [ ]:
rank_df + matrix_dot

In [ ]:
array_rank = np.array(rank)

In [ ]:
array_rank

In [ ]:
array_rank.sort(axis=0)

In [ ]:
array_rank
#array_rank[:,0]

In [ ]:
import pickle
with open('store_ranks.npy', 'wb') as writer:
    pickle.dump(ranks, writer)
    
with open('store_ranks.npy', 'rb') as reader:
    ranks3 = pickle.load(reader)
collections.Counter(ranks3)

# searching song lyrics

In [ ]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
#import nltk
#from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import scipy
#import spacy

import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
def remove_stopwords(texts):
    #stop_words = stopwords.words('english')
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    #return [[word for word in doc.split(' ') if word not in stop_words] for doc in texts]
    return [[word for word in doc.split(' ')] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# searching songs using lyrics ,song name and d2v

In [29]:
import requests

def filter_out_words(sentence):
    base_url = 'https://ZyFly.lib.id/lyricsMatch@dev/'
    params = {'sentence': sentence}
    response = requests.get(base_url, params)
    data = response.json()
    return data

what the boy


In [30]:
# read lyrics.csv
def process_corpus(column):
    data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])
    data = data[~data['lyrics'].isna()]
    data['lyrics'] = list(map(filter_out_words, data.lyrics))
    
    #data_words_nostops = remove_stopwords(data.lyrics)
    #data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    #corpus = list(map(lambda words: " ".join(words), data_lemmatized))
    
    count_vect = CountVectorizer().fit(data[column].values)
    words_count = count_vect.transform(data[column].values)
    tf_transformer = TfidfTransformer(sublinear_tf=True).fit(words_count)
    words_tfidf = tf_transformer.transform(words_count)
    return data, count_vect, tf_transformer, words_tfidf
    #scipy.sparse.save_npz('sparse_matrix.npz',words_tfidf) 
    #words_tfidf = scipy.sparse.load_npz('sparse_matrix.npz')

In [31]:
def process_query(count_vect, tf_transformer, lyric):
    #lyric_no_stops = remove_stopwords(lyric)
    #lyric_lemmatized = lemmatization(lyric_no_stops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    ##lyric_lemmatized = lemmatize_more(lyric_lemmatized)
    #lyric = list(map(lambda words: " ".join(words), lyric_lemmatized))
    words_count = count_vect.transform(lyric)
    search_tfidf = tf_transformer.transform(words_count)
    return search_tfidf

def search_top5(words_tfidf, search_tfidf, data):  
    matrix_dot = np.dot(words_tfidf, search_tfidf[0].transpose())
    #print(type(matrix_dot.todense()))
    matrix_dot = matrix_dot.todense()
    #sorted_index = matrix_dot.argsort(axis=0)
    #sorted_index = np.array(sorted_index)[:-6:-1]
    return matrix_dot
    #for iindex in sorted_index:
    #    print(data.iloc[iindex].song)
    #print(sorted_index)

In [ ]:
data, count_vect_lyric, tf_transformer_lyric, words_tfidf_lyric = process_corpus('lyrics')
data, count_vect_name, tf_transformer_name, words_tfidf_name = process_corpus('song')

In [25]:
#lyric = ['But I am much too full of resentment']
#lyric = ['I fall apart down to my core surprised when you caught me off my guard']
#lyric = ['She fooled me twice and its all my fault yeah She cut too deep  now she left me scarred']
#lyric = ['resentment']
#lyric = ['my fault']
#lyric = ['Ive been shattered My brains been battered']
lyric = ['Overwhelming Everything about you is so overwhelming']
#lyric = ['I think I got one Her soul is presidential like Barack']
#lyric = ['To the left  to the left To the left  to the left']
search_tfidf_lyric = process_query(count_vect_lyric, tf_transformer_lyric, lyric)
matrix_lyric = search_top5(words_tfidf_lyric, search_tfidf_lyric, data)

search_tfidf_name = process_query(count_vect_name, tf_transformer_name, lyric)
matrix_name = search_top5(words_tfidf_name, search_tfidf_name, data)

In [26]:
fname = get_tmpfile("my_doc2vec_model")
model = Doc2Vec.load(fname)

In [27]:
inferred_vector = model.infer_vector(lyric[0].split(' '))
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
rank = [[data.index.get_loc(int(docid.strip('doc'))), sim] for docid, sim in sims]

rank_df = pd.DataFrame(rank, columns=['docid', 'sim'])
rank_df = rank_df.sort_values(by=['docid'])
rank_df.index = rank_df.docid
rank_df = rank_df.drop(columns=['docid'])

matrix_d2v = rank_df.as_matrix()

In [28]:
# output is in a dict. 5 items. key varies but always different. each item is a list. list[0] sone name, list[1] singer name, list[2] youtube link
matrix_dot = 0.7*matrix_lyric + 0.1*matrix_name + 0.2*matrix_d2v
sorted_index = matrix_dot.argsort(axis=0)
sorted_index = np.array(sorted_index)[:-6:-1]
result_dict = dict()
link = 'https://www.youtube.com/results?search_query='
for iindex in sorted_index:
    parameter = data.iloc[iindex].song.values[0].replace(' ', '+') + '+' + data.iloc[iindex].singer.values[0].replace(' ', '+')
    print(link+parameter.lower())
    result = []
    result.append(data.iloc[iindex].song.values[0])
    result.append(data.iloc[iindex].singer.values[0])
    result.append(link+parameter.lower())
    result_dict[iindex[0]] = result
    
#    print(data.iloc[iindex].song)

https://www.youtube.com/results?search_query=overwhelming+jon+bellion
https://www.youtube.com/results?search_query=lay+me+down+-+single+version+sam+smith
https://www.youtube.com/results?search_query=lay+me+down+sam+smith
https://www.youtube.com/results?search_query=hand+of+god+(outro)+jon+bellion
https://www.youtube.com/results?search_query=betray+and+degrade+seether


# end here

In [ ]:
matrix_lyric[np.isnan(matrix_lyric)]

In [ ]:
#nltk.download('stopwords')
stop_words = stopwords.words('english')

In [ ]:
def remove_stopwords(texts):
    #return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return [[word for word in doc.split(' ') if word not in stop_words] for doc in texts]

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#def lemmatize_more(data):
#    tt = []
#    for tokens in data:
#        dd = [get_lemma(token) for token in tokens if len(token) > 4]
#        tt.append(dd)
#    return tt


In [ ]:
# Remove Stop Words
import spacy
data_words_nostops = remove_stopwords(data.lyrics)

In [ ]:
#data_lemmatized = lemmatize_more(data_lemmatized)

In [ ]:
corpus = list(map(lambda words: " ".join(words), data_lemmatized))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer().fit(data.lyrics.values)

In [ ]:
words_count = count_vect.transform(data.lyrics.values)

In [ ]:
tf_transformer = TfidfTransformer(sublinear_tf=True).fit(words_count)
words_tfidf = tf_transformer.transform(words_count)

In [ ]:
import scipy
scipy.sparse.save_npz('sparse_matrix.npz',words_tfidf) 
words_tfidf = scipy.sparse.load_npz('sparse_matrix.npz')

In [ ]:
#lyric = ['I fall apart down to my core surprised when you caught me off my guard']
#lyric = ['She fooled me twice and its all my fault yeah She cut too deep  now she left me scarred']
#lyric = ['Ive been shattered My brains been battered']
#lyric = ['Overwhelming Everything about you is so overwhelming']
#lyric = ['I think I got one Her soul is presidential like Barack']
lyric = ['But I am much too full of resentment']
#lyric_no_stops = remove_stopwords(lyric)
#lyric_lemmatized = lemmatization(lyric_no_stops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#lyric_lemmatized = lemmatize_more(lyric_lemmatized)
#lyric = list(map(lambda words: " ".join(words), lyric_lemmatized))

In [ ]:
words_count = count_vect.transform(lyric)
search_tfidf = tf_transformer.transform(words_count)

In [ ]:
similar_result = pd.DataFrame(index=data.index, columns=['sim'])

In [ ]:
#print(np.dot(words_tfidf, search_tfidf[0].transpose()))
#print(words_tfidf.shape)
#print(search_tfidf[0].transpose().shape)

matrix_dot = np.dot(words_tfidf, search_tfidf[0].transpose())
#print(matrix_dot.shape)
print(type(matrix_dot.todense()))
matrix_dot = matrix_dot.todense()
sorted_index = matrix_dot.argsort(axis=0)
sorted_index = np.array(sorted_index)[:-6:-1]
print(sorted_index)
#print(matrix_dot[6376])

In [ ]:
data.iloc[sorted_index[0]]

In [ ]:
#for pr_index, pr in data.iloc[:10000,].iterrows():
for pr_index, pr in data.iterrows():
    reviewed_iloc = data.index.get_loc(pr_index)
    #print(reviewed_iloc)
    if len(np.dot(search_tfidf[0], words_tfidf[reviewed_iloc].transpose()).tocoo().data) == 1: 
        similar_score = np.dot(search_tfidf[0], words_tfidf[reviewed_iloc].transpose()).tocoo().data[0] #if len(np.dot(words_tfidf[pr_iloc], words_tfidf[i].transpose()).tocoo().data) == 1]
        similar_result.loc[pr_index, 'sim'] = similar_score

In [ ]:
lyric

In [ ]:
similar_result.sort_values(by=['sim'], ascending=False)

In [ ]:
a = np.array([[1,4], [3,1]])

In [ ]:
data = pd.read_csv('lyrics.csv', names=['singer', 'song', 'lyrics'])
data = data[~data['lyrics'].isna()]

In [ ]:
s_index = data.index.get_loc(6383)
print(s_index)
print(data.iloc[s_index])


In [ ]:
similar_result.loc[6399]

In [ ]:
similar_result[similar_result['sim'].isna()].shape

# song name search

In [ ]:
count_vect = CountVectorizer().fit(corpus)
words_count = count_vect.transform(corpus)
tf_transformer = TfidfTransformer(sublinear_tf=True).fit(words_count)
words_tfidf = tf_transformer.transform(words_count)

# trying fuzzy string comparison

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]
process.extract("new york jets", choices, limit=2)

In [ ]:
data.loc[6343]

In [ ]:
fuzz.ratio("Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams", 'Shattered  shattered Love and hope and sex and dreams Are still surviving on the street Look at me  Im in tatters Im a shattered Shattered  Friends are so alarming And my lovers never charming Lifes just a cocktail party on the street Big Apple People dressed in plastic bags Directing traffic Some kind of fashion Shattered  Laughter  joy  and loneliness and sex and sex and sex and sex Look at me  Im in tatters Im a shattered Shattered  All this chitter-chatter  chitter-chatter  chitter-chatter about Shmatta  shmatta  shmatta  I cant give it away on 7th Avenue This towns been wearing tatters shattered  shattered Work and work for love and sex Aint you hungry for success  success  success  success Does it matter? Shattered Does it matter? Im shattered Shattered  Ahhh  look at me  Im a shattered Im a shattered Look at me  Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams and still surviving on the street And look at me  Im in tatters  yeah Ive been battered  what does it matter Does it matter  uh-huh Does it matter  uh-huh  Im a shattered Dont you know the crime rate is going up  up  up  up  up To live in this town you must be tough  tough  tough  tough  tough! You got rats on the west side Bed bugs uptown What a mess this towns in tatters Ive been shattered My brains been battered  splattered all over Manhattan  Uh-huh  this towns full of money grabbers Go ahead  bite the Big Apple  dont mind the maggots  huh Shadoobie  my brains been battered My friends they come around they Flatter  flatter  flatter  flatter  flatter  flatter  flatter Pile it up  pile it high on the platter')

In [ ]:
import regex
result = regex.match('(Im a yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams){e<=1}', 'Shattered  shattered Love and hope and sex and dreams Are still surviving on the street Look at me  Im in tatters Im a shattered Shattered  Friends are so alarming And my lovers never charming Lifes just a cocktail party on the street Big Apple People dressed in plastic bags Directing traffic Some kind of fashion Shattered  Laughter  joy  and loneliness and sex and sex and sex and sex Look at me  Im in tatters Im a shattered Shattered  All this chitter-chatter  chitter-chatter  chitter-chatter about Shmatta  shmatta  shmatta  I cant give it away on 7th Avenue This towns been wearing tatters shattered  shattered Work and work for love and sex Aint you hungry for success  success  success  success Does it matter? Shattered Does it matter? Im shattered Shattered  Ahhh  look at me  Im a shattered Im a shattered Look at me  Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams and still surviving on the street And look at me  Im in tatters  yeah Ive been battered  what does it matter Does it matter  uh-huh Does it matter  uh-huh  Im a shattered Dont you know the crime rate is going up  up  up  up  up To live in this town you must be tough  tough  tough  tough  tough! You got rats on the west side Bed bugs uptown What a mess this towns in tatters Ive been shattered My brains been battered  splattered all over Manhattan  Uh-huh  this towns full of money grabbers Go ahead  bite the Big Apple  dont mind the maggots  huh Shadoobie  my brains been battered My friends they come around they Flatter  flatter  flatter  flatter  flatter  flatter  flatter Pile it up  pile it high on the platter')

In [ ]:
choices = data.lyrics.values
%time process.extract("Im a shattered  yeah  Pride and joy and greed and sex Thats what makes our town the best Pride and joy and dirty dreams", choices, limit = 5)

In [ ]:
Counter(reviewers_semantic_sim['rchande'].isnull())